## Setup and Import Libraries

In [2]:
import os
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_openai_tools_agent, AgentExecutor
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
llm = ChatGroq(model_name='llama3-8b-8192')
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Inbuilt Tools Creation 

In [7]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
wikipedia_tool.name

'wikipedia'

In [8]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv_tool = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv_tool.name

'arxiv'

### Combining Tools

In [9]:
tools = [wikipedia_tool, arxiv_tool]

# tools

## Custom Tools Creation

In [10]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [11]:
vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)

In [12]:
retriever = vector_store.as_retriever()

In [13]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='langsmith_search',
    description="Search any information about langsmith")

retriever_tool.name

'langsmith_search'

In [14]:
tools = [wikipedia_tool, arxiv_tool, retriever_tool]

# tools

## Agent

### Prompt Template from Hub

In [15]:
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Create Agent

In [16]:
agent = create_openai_tools_agent(
    llm=llm, prompt=prompt, tools=tools
)

# agent

### Agent Executor

In [17]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

# agent_executor

In [19]:
# query = "Tell me about Langsmith"
# query = "Who won the Fifa World Cup 2022"
query = "What is the paper arXiv:1810.04805 about?"

agent_executor.invoke(
    {'input': query}
)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1810.04805'}`


Published: 2019-05-24
Title: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Authors: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova
ntroduce a new language representation model called BERT,
Invoking: `arxiv` with `{'query': '1810.04805'}`


Published: 2019-05-24
Title: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Authors: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova
The paper "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova has already been discussed. It seems that there is no additional information about the paper "arXiv:1810.04805" that I can provide.

> Finished chain.


{'input': 'What is the paper arXiv:1810.04805 about?',
 'output': 'The paper "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" by Jacob Devlin, Ming-Wei Chang, Kenton Lee, and Kristina Toutanova has already been discussed. It seems that there is no additional information about the paper "arXiv:1810.04805" that I can provide.'}